In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import prince


df1=pd.read_csv('tracks_features.csv')
df_popu=pd.read_csv('tracks_features_popu_100_pourcents.csv')

# 1) Méthode KMeans

## a) Algorithme KMeans

In [ ]:
# preprocessing

# supression des données inutiles au clustering

df1=df1.set_index('id')
df_cluster=df1.drop(['name','album','album_id','artists',
                     'artist_ids','track_number','disc_number',
                     'time_signature','year','release_date'],axis=1)

# Application d'un scaler

scaler=MinMaxScaler()
scaler.fit(df_cluster)
df_cluster_scaled=pd.DataFrame(scaler.transform(df_cluster),index=df_cluster.index,columns=df_cluster.columns)
df_cluster_scaled.head()


In [ ]:
# application méthode du coude pour trouver le nombre de cluster à paramétrer

k_value=[2,3,4,5,6,7,8,9,10,11,12]

disto=[]

for k in k_value:
    clusters=KMeans(n_clusters=k)
    clusters.fit(df_cluster_scaled)
    disto.append(sum(np.min(cdist(df_cluster_scaled,clusters.cluster_centers_,'euclidean'),axis=1))/np.size(df_cluster_scaled,axis=0))


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(k_value,disto)
plt.axvline(x=3,color='red',label='3 clusters')
plt.axvline(x=5,color='green',label='5 clusters')
plt.legend()

On retient 5 clusters

In [ ]:
# on effectue le clustering avec 5 clusters et la méthode du KMeans

kmeans=KMeans(n_clusters=5)
kmeans.fit(df_cluster_scaled)

labels=kmeans.labels_

In [ ]:
# On observe la répartition des labels

pd.DataFrame(labels).value_counts()

In [ ]:
sns.countplot(labels).set_title('nombre de morceaux par clusters')

## b) Application de la PCA sklearn

In [ ]:
# on applique la PCA

pca1=PCA(n_components=3)
pca1.fit(df_cluster_scaled)
df_pca1=pca1.transform(df_cluster_scaled)
liste_pca=pca1.explained_variance_ratio_

In [ ]:
# affichage du cercle de corrélation (affiche sur les axes PCA 1 et PCA 2)

(fig, ax) = plt.subplots(figsize=(12, 12))
for i in range(0, pca1.components_.shape[1]):
    ax.arrow(0,
             0,  # Start the arrow at the origin
             pca1.components_[0, i]*0.95,  #0 for PC1
             pca1.components_[1, i]*0.95,  #1 for PC2
             head_width=0.05,
             head_length=0.05,
             color='b',
             alpha=0.5)

    plt.text(pca1.components_[0, i] ,
             pca1.components_[1, i] ,
             df_cluster_scaled.columns.values[i]
             ,fontsize='large')


an = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale
plt.axis('equal')
plt.plot([-1, 1], [0, 0], color='silver', linestyle='-', linewidth=1)
plt.plot([0, 0], [-1, 1], color='silver', linestyle='-', linewidth=1)
ax.set_title('Cercle de corrélation du module sklearn (classique) PCA 1 & 2',fontsize='xx-large')
plt.xlabel('PCA 1 - '+str(np.round(liste_pca[0]*100,2))+' %')
plt.ylabel('PCA 2 - '+str(np.round(liste_pca[1]*100,2))+' %')

plt.plot([0.25,-0.25,-0.25,0.25,0.25],[0.05,0.05,-0.05,-0.05,0.05],'r',alpha=0.9)

plt.annotate('',xy=(0.25,0),xytext=(1.4,0),arrowprops={'facecolor':'red'})
plt.axes([1,0.3,0.5,0.5])
plt.axis([-0.25,0.25,-0.05,0.05])
for i in range(0, pca1.components_.shape[1]):
    print(pca1.components_[0,i],pca1.components_[1,i],df_cluster_scaled.columns.values[i],abs(pca1.components_[0,i])<0.25)
    
    plt.arrow(0,
             0,  # Start the arrow at the origin
             pca1.components_[0, i]*0.95,  #0 for PC1
             pca1.components_[1, i]*0.95,  #1 for PC2
             head_width=0.005,
             head_length=0.03,
             color='b',
             alpha=0.5)
    if abs(pca1.components_[0,i])<0.25:
        if abs(pca1.components_[1,i])<0.05:
             plt.text(pca1.components_[0, i] ,
                     pca1.components_[1, i] ,
                     df_cluster_scaled.columns.values[i],
                     fontsize='large')


plt.title("Zoom",fontsize='xx-large')

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# affichage du cercle de corrélation (affiche sur les axes PCA 1 et PCA 3)

(fig, ax) = plt.subplots(figsize=(12, 12))
for i in range(0, pca1.components_.shape[1]):
    ax.arrow(0,
             0,  # Start the arrow at the origin
             pca1.components_[0, i]*0.95,  #0 for PC1
             pca1.components_[2, i]*0.95,  #1 for PC3
             head_width=0.03,
             head_length=0.03,
             color='b',
             alpha=0.5)

    plt.text(pca1.components_[0, i],
             pca1.components_[2, i],
             df_cluster_scaled.columns.values[i])


an = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale
plt.axis('equal')
plt.plot([-1, 1], [0, 0], color='silver', linestyle='-', linewidth=1)
plt.plot([0, 0], [-1, 1], color='silver', linestyle='-', linewidth=1)
ax.set_title('Cercle de corrélation du module sklearn (classique) PCA 1 & 3')
plt.xlabel('PCA 1 - '+str(np.round(100*liste_pca[0],2))+' %')
plt.ylabel('PCA 3 - '+str(np.round(100*liste_pca[2],2))+' %')

plt.show()

In [ ]:
# affichage PCA en 3 dimensions

fig=plt.figure(figsize=(10,10))
ax=fig.gca(projection='3d')
ax.scatter(df_pca1[:,0],df_pca1[:,1],df_pca1[:,2],c=labels,cmap=plt.cm.Spectral,s=0.1)
ax.set_xlabel('PCA 1 - '+str(np.round(liste_pca[0]*100,2))+' %')
ax.set_ylabel('PCA 2 - '+str(np.round(liste_pca[1]*100,2))+' %')
ax.set_zlabel('PCA 3 - '+str(np.round(liste_pca[2]*100,2))+' %')
plt.title("PCA 3 dimensions module classique")
plt.show()

In [ ]:
# affichage PCA en 2 dimensions (PCA 1 et PCA 2)

fig=plt.figure(figsize=(10,10))
plt.scatter(df_pca1[:,0],df_pca1[:,1],c=labels,cmap=plt.cm.Spectral,s=0.1)
plt.xlabel('PCA 1 - '+str(np.round(liste_pca[0]*100,2))+' %')
plt.ylabel('PCA 2 - '+str(np.round(liste_pca[1]*100,2))+' %')
plt.title("PCA 2 dimensions module classique")
plt.show()

In [ ]:
# affichage PCA en 2 dimensions (PCA 2 et PCA 3)

fig=plt.figure(figsize=(10,10))
plt.scatter(df_pca1[:,1],df_pca1[:,2],c=labels,cmap=plt.cm.Spectral,s=0.1)
plt.xlabel('PCA 2 - '+str(np.round(liste_pca[1]*100,2))+' %')
plt.ylabel('PCA 3 - '+str(np.round(liste_pca[2]*100,2))+' %')
plt.title("PCA 2 dimensions module classique")
plt.show()

## c) PCA module prince

In [ ]:
# application PCA module prince

pca2=prince.PCA(n_components=3)
pca2=pca2.fit(df_cluster_scaled)
df_pca2=pca2.transform(df_cluster_scaled)
corr = pca2.column_correlations(df_cluster_scaled)

In [ ]:
#liste des ratios explicatif de la pca

liste_pca2=pca2.explained_inertia_

In [ ]:
# affichage cercle de corrélation (PCA 1 et PCA 2)

fig, axes = plt.subplots(figsize=(10, 10))
axes.set_xlim(-1, 1)
axes.set_ylim(-1, 1)
# Affichage des variables
for j in df_cluster_scaled.columns:
    plt.annotate(j, (corr.loc[j, 0], corr.loc[j, 1]), color='#091158')
    plt.arrow(0, 0, corr.loc[j, 0]*0.95, corr.loc[j, 1]*0.95, alpha=0.5, head_width=0.03, color='b')
# Ajout des axes
plt.plot([-1, 1], [0, 0], color='silver', linestyle='-', linewidth=1)
plt.plot([0, 0], [-1, 1], color='silver', linestyle='-', linewidth=1)
# Cercle et légendes
cercle = plt.Circle((0, 0), 1, color='#16E4CA', fill=False)
axes.add_artist(cercle)
plt.xlabel('PCA 1 - '+str(np.round(liste_pca2[0]*100,2))+' %')
plt.ylabel('PCA 2 - '+str(np.round(liste_pca2[1]*100,2))+' %')
plt.title('Cercle de correlation module prince PCA 1 & 2');

In [ ]:
# affichage cercle de corrélation (PCA 1 et 3)

fig, axes = plt.subplots(figsize=(10, 10))
axes.set_xlim(-1, 1)
axes.set_ylim(-1, 1)
for j in df_cluster_scaled.columns:
    plt.annotate(j, (corr.loc[j, 0], corr.loc[j, 2]), color='#091158')
    plt.arrow(0, 0, corr.loc[j, 0]*0.95, corr.loc[j, 2]*0.95, alpha=0.5, head_width=0.03, color='b')
# Ajout des axes
plt.plot([-1, 1], [0, 0], color='silver', linestyle='-', linewidth=1)
plt.plot([0, 0], [-1, 1], color='silver', linestyle='-', linewidth=1)
# Cercle et légendes
cercle = plt.Circle((0, 0), 1, color='#16E4CA', fill=False)
axes.add_artist(cercle)
plt.xlabel('PCA 1 - '+str(np.round(liste_pca2[0]*100,2))+' %')
plt.ylabel('PCA 3 - '+str(np.round(liste_pca2[2]*100,2))+' %')
plt.title('Cercle de correlation module prince PCA 1 & 3');

In [ ]:
# On affiche la PCA en 3 dimensions

fig=plt.figure(figsize=(10,10))
ax=fig.gca(projection='3d')
ax.scatter(df_pca2[0],df_pca2[1],df_pca2[2],c=labels,cmap=plt.cm.Spectral,s=0.1)
ax.set_xlabel('PCA 1 - '+str(np.round(liste_pca2[0]*100,2))+' %')
ax.set_ylabel('PCA 2 - '+str(np.round(liste_pca2[1]*100,2))+' %')
ax.set_zlabel('PCA 3 - '+str(np.round(liste_pca2[2]*100,2))+' %')
plt.title("PCA 3 dimensions module Prince")
plt.show()

In [ ]:
# On affiche la PCA en 2 dimensions (PCA 1 et PCA 2)

plt.figure(figsize=(10,10))
plt.scatter(df_pca2[0],df_pca2[1],c=labels,cmap=plt.cm.Spectral,s=0.1)
plt.xlabel('PCA 1 - '+str(np.round(liste_pca2[0]*100,2))+' %')
plt.ylabel('PCA 2 - '+str(np.round(liste_pca2[1]*100,2))+' %')
plt.title('PCA 2 dimensions du module Prince')
plt.show()

In [ ]:
# On affiche la PCA en 2 dimensions (PCA 2 et PCA 3)

plt.figure(figsize=(10,10))
plt.scatter(df_pca2[1],df_pca2[2],c=labels,cmap=plt.cm.Spectral,s=0.1)
plt.xlabel('PCA 2 - '+str(np.round(liste_pca2[1]*100,2))+' %')
plt.ylabel('PCA 3 - '+str(np.round(liste_pca2[2]*100,2))+' %')
plt.title('PCA 2 dimensions du module Prince')
plt.show()

Les résulats de cette PCA ne sont pas bons, on conservera les résultats de la première PCA

## d) Analyse clustering

In [ ]:
# on joint les labels aux nom de morceaux d'artistes et d'album

label_pres=df1[['name','album','artists']].merge(right=pd.DataFrame(labels,index=df_cluster_scaled.index).rename(columns={0:'labels'}),on='id')
label_pres.head(30)

In [ ]:
# création d'une table du nombre de clusters par artistes

df_artists=label_pres.groupby('artists').agg({'labels':['nunique']})

In [ ]:
# on affiche le nombre d'artiste par le nombre de clusters distincts

sns.countplot(df_artists[('labels','nunique')]).set(xlabel="Nombre de cluster distincts",ylabel="Nombre d'artistes")

In [ ]:
# pourcentage du nombre d'artiste par nombre de clusters distincts

tot=df_artists['labels'].count()[0]
per=100*df_artists['labels'].value_counts()/tot
per